# Reading and storing Pyspark dataframe to InfluxDB

In [1]:
!pip install influxdb_client influxdb

## Reading the excel to DataFrame then converting it to Spark Dataframe

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("Test").getOrCreate()
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'])
sdf = spark.createDataFrame(df) 
sdf.show(5)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+----------+----------+------+--------+-----------+-----------+----------+--------+----------+----------+--------+--------+--------+--------+-----+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+----------+----------+----------+----------+----------+------+------+------+------+--------+-----------+-----------+-------+----------+-----------+----------+--------+--------+-----------+-----------+-----------+-----------+-----+----------+----------+----------+----------+-----------+-----------+-----------+----------+------+------+------+------+--------+-----------+-----------+----------+----------+----------+----------+-------+-------+--------+--------+--------+-------+--------+-----------+-----------+-----------+
|              GMT +0|   FIT 101|   LIT 101|MV 101|P1_STATE|P101 Status|P102 Status|   AIT 201| AIT 202|   AIT 203|   FIT 201|  LS 201|  LS 202| LSL 203|LSLL 203|MV201|P2_STATE|P201 Status|P202 S

In [49]:
# import influxdb_client
# from influxdb_client.client.write_api import SYNCHRONOUS

# token = 'Qd6R1JJWOOKfcKmBn0s50wHLqA-2LjLjEGJiUAGp1_RCes_JRe_bkELLYL5jqCXRQ1Haww3_mrHk_69yN88L0A=='
# org = 'ELTE'
# bucket = 'ELTE'

# with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
#     write_api = client.write_api(write_options=SYNCHRONOUS)
#     dataCollect = sdf.collect()
#     for row in dataCollect:
#         p = influxdb_client.Point("swat_measure").field('LIT 101', row['LIT 101'])
#         write_api.write(bucket, org, p)
    
    

In [16]:
attack1 = np.array(sdf.select("FIT 401").collect())
print(morris(attack1))

25610.981454890723


In [28]:
sampling1 = ReservoirSampling(20)
for elem in attack1:
    sampling1.update(elem)
sampling1.ret()

array([0.80376464, 0.799411  , 0.7972341 , 0.8049171 , 0.7973622 ,
       0.799411  , 0.8076061 , 0.79492927, 0.7991549 , 0.80222803,
       0.79966706, 0.8041488 , 0.8031244 , 0.8015878 , 0.7964658 ,
       0.799411  , 0.80171585, 0.7960817 , 0.79966706, 0.80517316])

## Sending data to InfluxDB

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

token = 'Qd6R1JJWOOKfcKmBn0s50wHLqA-2LjLjEGJiUAGp1_RCes_JRe_bkELLYL5jqCXRQ1Haww3_mrHk_69yN88L0A=='
org = 'ELTE'
bucket = 'ELTE'

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for row in dataCollect:
        for col in sdf.columns[1:]:
            p = influxdb_client.Point("swat_measure_2").field(col, row[col])
            write_api.write(bucket, org, p)    

## Reservoir Sampling

In [27]:
import numpy as np
import random
    
class ReservoirSampling:
    def __init__(self, size):
        self.arr = np.zeros(size)
        self.count = 0
    def update(self,elem):
        # if counter less than the pool size we add this element in the first free position 
        if self.count < len(self.arr):
            for i in range(len(self.arr)):
                if self.arr[i]==0:
                    self.arr[i] = elem
                    self.count += 1
        # it counter >= pool size then we have to apply update algorithm
        r = random.randint(0, self.count-1)
        if r < len(self.arr):
            self.arr[r]=elem
        self.count += 1
    
    def ret(self):
        return self.arr
               

In [15]:
def morris(arr):
    cont = 0
    c = 10
    for elem in arr:
        cont += 2**(-c)
    return (2**cont)-1